# Reading and using BWA

##Preprocessing

The command !apt-get install -y bwa installs the BWA (Burrows-Wheeler Aligner) software on your system, enabling you to perform sequence alignment of reads against a reference genome.

In [1]:
# Install BWA
!apt-get install -y bwa

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  samtools
The following NEW packages will be installed:
  bwa
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 195 kB of archives.
After this operation, 466 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 bwa amd64 0.7.17-6 [195 kB]
Fetched 195 kB in 0s (438 kB/s)
Selecting previously unselected package bwa.
(Reading database ... 123621 files and directories currently installed.)
Preparing to unpack .../bwa_0.7.17-6_amd64.deb ...
Unpacking bwa (0.7.17-6) ...
Setting up bwa (0.7.17-6) ...
Processing triggers for man-db (2.10.2-1) ...


Unzips the Reference file

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Unzip the uploaded file
!unzip "/content/drive/MyDrive/IP/Meeting 4: 23 09 2024/mycobacterium_tuberculosis_h37rv_2_data.zip" -d "/content/"

unzip:  cannot find or open /content/drive/MyDrive/IP/Meeting 4: 23 09 2024/mycobacterium_tuberculosis_h37rv_2_data.zip, /content/drive/MyDrive/IP/Meeting 4: 23 09 2024/mycobacterium_tuberculosis_h37rv_2_data.zip.zip or /content/drive/MyDrive/IP/Meeting 4: 23 09 2024/mycobacterium_tuberculosis_h37rv_2_data.zip.ZIP.


In [4]:
# List the files in the current directory
!ls -lh

total 8.0K
drwx------ 6 root root 4.0K Oct 11 14:11 drive
drwxr-xr-x 1 root root 4.0K Oct  8 13:26 sample_data


The output of the `!ls -lh` command lists the files and directories in the current working directory with detailed information. Here's a breakdown of each line:

 General Information

-   **Total 12MB**: The total size of all files in the current directory is 12 Megabytes.

 Summary of Differences

| **Feature** | **Contigs (`contigs.fasta`)** | **Supercontigs (`supercontigs.fasta`)** |
| --- | --- | --- |
| **Definition** | Continuous sequences assembled from reads | Linked contigs with known order/orientation |
| **Length** | Shorter | Longer |
| **Gaps** | No gaps | May contain gaps (`N` bases) |
| **Assembly Level** | Initial stage of assembly | Higher level, linked contigs (scaffolds) |
| **Use Cases** | Local sequence analysis | Genome structure and gene prediction |
| **Completeness** | Less complete representation of genome | More complete, near full-chromosome length |

Installing both `bwa` and `samtools` together allows you to perform a complete workflow for analyzing sequencing data:

1.  **Align Reads**: Use `bwa` to align your sequencing reads to a reference genome.
2.  **Process Alignments**:
    -   Convert the resulting SAM file to BAM format using `samtools`.
    -   Sort and index the BAM file for efficient access and analysis.
    -   Use `samtools` to generate summary statistics, view alignments, and prepare data for variant calling or visualization.

-   **Indexing**: Preprocessing the reference genome to create data structures (BWT, SA) that enable fast searching and alignment.
-   **Example**: A read is aligned to the reference using the index, which helps BWA quickly find the correct position without scanning the entire genome.
-   **Efficiency**: Indexing significantly speeds up the alignment process by reducing the number of comparisons and memory usage.

In [5]:
# Install BWA and samtools
!apt-get install -y bwa samtools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bwa is already the newest version (0.7.17-6).
The following additional packages will be installed:
  libhts3 libhtscodecs2
Suggested packages:
  cwltool
The following NEW packages will be installed:
  libhts3 libhtscodecs2 samtools
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 963 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhtscodecs2 amd64 1.1.1-3 [53.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhts3 amd64 1.13+ds-2build1 [390 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 samtools amd64 1.13-4 [520 kB]
Fetched 963 kB in 1s (679 kB/s)
Selecting previously unselected package libhtscodecs2:amd64.
(Reading database ... 123638 files and directories currently installed.)
Preparing to unpack .../libhtscodecs2_1.1.1-3_amd64

In [6]:
# index the reference
!bwa index mycobacterium_tuberculosis_h37rv_2_contigs.fasta

[bwa_idx_build] fail to open file 'mycobacterium_tuberculosis_h37rv_2_contigs.fasta' : No such file or directory


Here's a brief description of each file:

1.  **`.fasta.amb`**: This file contains information about ambiguous bases in the reference genome. It helps the aligner handle ambiguous nucleotide sequences (like N's) efficiently.

2.  **`.fasta.ann`**: This file is an annotation file that contains information about the sequences, including their names and lengths.

3.  **`.fasta.bwt`**: This is the Burrows-Wheeler Transform index file. It is used to efficiently align reads to the reference genome. The BWT is a reversible transformation that enables efficient data compression and search.

4.  **`.fasta.pac`**: This file contains the packed version of the reference sequence, stored in a compressed binary format to reduce the file size.

5.  **`.fasta.sa`**: This is the suffix array index file. It is used in conjunction with the `.bwt` file to efficiently locate substrings in the reference sequence during alignment.

The following code aligns sequencing reads from the FASTQ file to the reference genome using the `bwa mem` algorithm and outputs the alignments in a SAM file.

What Happens During the Execution ?

1.  **Alignment Process**:

    -   The `bwa mem` algorithm reads the input FASTQ file (`SRR30683486.fastq.gz`) and aligns each read to the reference genome (`mycobacterium_tuberculosis_h37rv_2_supercontigs.fasta`).
    -   It uses the index files created from the reference genome (generated using the `bwa index` command) to find the best alignment positions quickly and efficiently.
2.  **Output in SAM Format**:

    -   The aligned reads, along with detailed alignment information (position, quality, mismatches, etc.), are saved in the `aligned_reads.sam` file.
    -   This SAM file is a standard format used to store alignment information, and it can be converted to BAM format, sorted, and indexed for further analysis.

In [7]:
# Align the reads with the reference genome using bwa mem
!bwa mem /content/mycobacterium_tuberculosis_h37rv_2_contigs.fasta \
//content/drive/MyDrive/IP/Meeting\ 4:\ 23\ 09\ 2024/SRR30683486.fastq.gz \
> /content/aligned_reads.sam

[E::bwa_idx_load_from_disk] fail to locate the index files


## Reading the results of BWA

### What if we do not convert it to BAM?

In [8]:
!head -n 23 /content/aligned_reads.sam

The given line from the SAM file provides detailed alignment information for a single sequencing read (`SRR30683486.1`) against a reference sequence (`AL123456`). Here's a breakdown of each component:

 1\. **Header Lines**

These lines provide meta-information about the reference genome and alignment tools used.

-   **`@SQ SN:AL123456 LN:4411532`**:

    -   **`@SQ`**: Indicates a sequence dictionary entry.
    -   **`SN:AL123456`**: The reference sequence name, in this case, a contig or chromosome labeled `AL123456`.
    -   **`LN:4411532`**: The length of the reference sequence `AL123456` is 4,411,532 base pairs.
-   **`@PG ID:bwa PN:bwa VN:0.7.17-r1188 CL:bwa mem /content/mycobacterium_tuberculosis_h37rv_2_contigs.fasta /content/drive/MyDrive/Academic/CMI DS 23-25/IP/Meeting 4: 23 09 2024/SRR30683486.fastq.gz`**:

    -   **`@PG`**: Program information.
    -   **`ID:bwa`**: Program ID is `bwa`.
    -   **`PN:bwa`**: Program name is `bwa`.
    -   **`VN:0.7.17-r1188`**: Version of `bwa` used for alignment.
    -   **`CL:bwa mem ...`**: The command line (`CL`) used to run `bwa mem` with the reference file (`mycobacterium_tuberculosis_h37rv_2_contigs.fasta`) and the input read file (`SRR30683486.fastq.gz`).

 2\. **Alignment Line**

The main part of the SAM file, this line provides details about how a single read aligns to the reference genome.

**Line**:

For the first line only:

`SRR30683486.1	0	AL123456	454690	60	150M	*	0	0	GTTGCTGAGCATCGCCGCCTCCAGCGCCTTCGCCGAACACGCCGCGCCGGGCCTGATCGCGCTCTTCTCGTCTCGGGCCGACGACCTTTCGGTCGAGTTGAGCGTGCATCCCACCAGCCGGTTCCGCGAACTGATCTGCTCGCGCGCCGT	??????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????	NM:i:2	MD:Z:7C51A90	AS:i:140	XS:i:0`

| **Field** | **Description** | **Value** |
| --- | --- | --- |
| **QNAME** | Query name (read identifier). | `SRR30683486.1` |
| **FLAG** | Bitwise flag indicating the read's properties. | `0` (indicates the read is mapped to the forward strand) |
| **RNAME** | Reference sequence name where the read is aligned. | `AL123456` (reference sequence or contig name) |
| **POS** | 1-based leftmost position of alignment. | `454690` (the read starts aligning at position 454,690 on the reference sequence `AL123456`) |
| **MAPQ** | Mapping quality score (0-255). | `60` (high confidence in this alignment) |
| **CIGAR** | Describes the alignment in terms of matches, insertions, deletions, etc. | `150M` (150 bases match the reference exactly, no insertions/deletions) |
| **RNEXT** | Reference name of the mate/next read (if paired). | `*` (no mate information, or single-end read) |
| **PNEXT** | Position of the mate/next read (if paired). | `0` (no mate position, or single-end read) |
| **TLEN** | Observed template length (insert size). | `0` (no template length, or single-end read) |
| **SEQ** | Query sequence (the actual read sequence). | `GTTGCTGAGCATCGCCGCCTCCAGCGCCTTCGCCGAACACGCCGCGCCGGGCCTGATCGCGCTCTTCTCGTCTCGGGCCGACGACCTTTCGGTCGAGTTGAGCGTGCATCCCACCAGCCGGTTCCGCGAACTGATCTGCTCGCGCGCCGT` |
| **QUAL** | Phred-scaled base quality scores. | `??????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????` (indicates quality is missing or not available) |
| **Optional Tags** | Additional information about the alignment. | `NM:i:2`, `MD:Z:7C51A90`, `AS:i:140`, `XS:i:0` |

 Detailed Explanation of Key Fields

1.  **QNAME (Query Name)**:

    -   **`SRR30683486.1`**: The unique identifier for the read. This allows you to trace this specific read in your dataset.
2.  **FLAG**:

    -   **`0`**: Indicates the read is mapped to the forward strand with no special properties.
        -   `0` means the read is mapped and is a primary alignment.
3.  **RNAME (Reference Name)**:

    -   **`AL123456`**: The reference sequence or contig to which the read is aligned.
4.  **POS (Position)**:

    -   **`454690`**: The starting position of the read alignment on the reference sequence `AL123456`. This means the first base of the read aligns to position 454,690 of the reference.
5.  **MAPQ (Mapping Quality)**:

    -   **`60`**: Indicates a high confidence in the read alignment. A MAPQ of 60 is the maximum score, signifying the alignment is unique with very high certainty.
6.  **CIGAR (Compact Idiosyncratic Gapped Alignment Report)**:

    -   **`150M`**: This indicates that all 150 bases in the read match exactly to the reference genome without any insertions, deletions, or clipping.
        -   `150M` means 150 bases are aligned with either matches or mismatches but no gaps or clipping.
7.  **SEQ (Sequence)**:

    -   The sequence of the read that has been aligned to the reference.
    -   **`GTTGCTGAGCATCGCC...`**: This is the nucleotide sequence of the read.
8.  **QUAL (Quality Scores)**:

    -   The quality scores for each base in the read. Typically, these are ASCII-encoded Phred scores representing the confidence in each base call.
    -   **`????????????...`**: In this case, it appears as `?`, which may indicate missing or placeholder quality scores, suggesting the quality information is not present.
9.  **Optional Tags**:

    -   Provide additional details about the alignment:

    -   **`NM:i:2`**: Number of mismatches between the read and the reference sequence. Here, there are 2 mismatches.

    -   **`MD:Z:7C51A90`**: Mismatch descriptor tag:

        -   `7C51A90` means:
            -   `7`: First 7 bases match exactly.
            -   `C`: Mismatch at the 8th position (reference has `C`).
            -   `51`: Next 51 bases match exactly.
            -   `A`: Mismatch at the 60th position (reference has `A`).
            -   `90`: Last 90 bases match exactly.
    -   **`AS:i:140`**: Alignment score, a measure of how well the read aligns to the reference. Higher scores indicate better alignments.

    -   **`XS:i:0`**: Suboptimal alignment score. If there were multiple potential alignments for this read, this would show the score of the next best alignment. Here, `0` suggests no suboptimal alignment.

 Summary

This alignment line provides a detailed view of how the read `SRR30683486.1` aligns to the reference sequence `AL123456`:

-   The read aligns starting at position `454690` on the reference with a high-quality score of `60`.
-   The CIGAR string `150M` indicates that all 150 bases of the read match exactly to the reference sequence.
-   There are 2 mismatches between the read and the reference, as indicated by the `NM:i:2` tag.
-   The `MD:Z:7C51A90` tag specifies the exact positions of these mismatches.

This information is crucial for understanding the accuracy and quality of the alignment, which is necessary for downstream analyses like variant calling or gene expression studies. If you need more details or specific interpretations, feel free to ask!

In [9]:
# !cat /content/aligned_reads.sam

Total number of lines

In [10]:
!wc -l /content/aligned_reads.sam

0 /content/aligned_reads.sam


To find the total number of discrepancies (mismatches, insertions, and deletions) between the reads in your SAM file and the reference genome, you can use the samtools tool along with some additional processing.

In [11]:
!apt-get install -y samtools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
samtools is already the newest version (1.13-4).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [12]:
!samtools view -Sb /content/aligned_reads.sam > /content/aligned_reads.bam
#!samtools view -Sb /content/aligned_reads.sam > /content/aligned_reads.bam
!samtools view /content/aligned_reads.bam | awk '{for(i=12;i<=NF;i++) if($i ~ /^NM:i:/) {split($i,a,":"); total += a[3]}} END {print "Total number of discrepancies:", total}'

[main_samview] fail to read the header from "/content/aligned_reads.sam".
[main_samview] fail to read the header from "/content/aligned_reads.bam".
Total number of discrepancies: 


In [13]:
import pandas as pd

# Read the first 20 lines of the SAM file, skipping lines with parsing errors
sam_file = '/content/aligned_reads.sam'
sam_data = pd.read_csv(
    sam_file,
    sep='\t',
    comment='@',
    header=None,
    names=None,
    on_bad_lines='skip',  # Skip lines with mismatched columns  # Read the first 20 lines
)

# Display the DataFrame
sam_data

EmptyDataError: No columns to parse from file

In [ ]:
565488-286533*2

Miss Match

In [ ]:
sum = 0
for i in sam_data[11]:
  sum+= int(i[-1])
print(sum) #== #249797

# Reading and using Picard

### 1\. **Sorting and Merging BAM/SAM Files**

-   **Sorting**: Think of sorting as arranging your data in a particular order. For BAM/SAM files, this usually means ordering the reads based on their position in the genome. This makes it easier to find and analyze specific regions in your data.
-   **Merging**: If you have multiple files containing parts of your data, merging them combines all these parts into a single, complete file. This is useful when you have data from different experiments or runs and want to analyze them together.

### 2\. **Marking Duplicates**

-   During sequencing, sometimes the same piece of DNA gets copied multiple times. Picard can find these duplicate copies and mark them. By marking duplicates, you can ignore them in your analysis, so your results aren't skewed by counting the same piece of DNA more than once.

### 3\. **Collecting Quality Metrics**

-   Picard can check the quality of your sequencing data by calculating various statistics. For example, it can tell you the average size of DNA fragments, how well your reads align to the reference genome, and if there are any unusual patterns. This helps ensure your data is good enough for further analysis.

### 4\. **File Conversion**

-   Picard can change the format of your data files. For example, it can switch between BAM (a compressed, efficient format) and SAM (a plain-text version) formats. This is helpful when different tools require different formats.

### 5\. **Manipulating Headers**

-   The header of a BAM/SAM file contains important information about the data, like the sample name and sequencing details. Picard allows you to change this information, such as adding new details or correcting mistakes, so that your files have accurate and complete metadata.

Picard is a Java-based tool, so you need to install Java first.

In [ ]:
!apt-get install -y openjdk-11-jdk

You can download the Picard JAR file from the official Broad Institute website.

In [ ]:
!wget -O picard.jar https://github.com/broadinstitute/picard/releases/download/2.27.4/picard.jar

In [ ]:
!java -jar picard.jar SortSam \
    INPUT=/content/aligned_reads.bam \
    OUTPUT=/content/sorted_reads.bam \
    SORT_ORDER=coordinate

Key Points in the Log:

1.  **Command Line Syntax Change Notification**:

    -   Picard is notifying you that the command line syntax is changing. They are moving to a new syntax format. The example given in the log shows how the new command syntax would look:

        bash

        `SortSam -INPUT /content/aligned_reads.bam -OUTPUT /content/sorted_reads.bam -SORT_ORDER coordinate`

2.  **Library Loading**:

    -   `INFO NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/content/picard.jar!...`
    -   This indicates that Picard is loading native libraries required for compression and decompression operations, which are used to handle the BAM file format efficiently.
3.  **Execution Information**:

    -   The line `[Mon Sep 23 05:25:43 UTC 2024] SortSam INPUT=/content/aligned_reads.bam OUTPUT=/content/sorted_reads.bam ...`
    -   This shows the command being executed with various parameters:
        -   `INPUT`: The input BAM file.
        -   `OUTPUT`: The output file to which the sorted reads are written.
        -   `SORT_ORDER`: The order in which the reads are sorted (in this case, `coordinate`).
        -   `VERBOSITY=INFO`: The level of logging information.
        -   `VALIDATION_STRINGENCY=STRICT`: How strictly Picard checks the input file for errors.
        -   Other parameters control compression, indexing, memory usage, and more.
4.  **Non-Increasing Record Positions**:

    -   `INFO 2024-09-23 05:25:43 SortSam Seen many non-increasing record positions. Printing Read-names as well.`
    -   This message suggests that the input BAM file had records (reads) that were not in the expected order (which is why you're sorting them). This is typical when sorting BAM files.
5.  **Processing Complete**:

    -   `INFO 2024-09-23 05:25:51 SortSam Finished reading inputs, merging and writing to output now.`
    -   This indicates that Picard has finished reading the input BAM file, merged the reads in the correct order, and is now writing the sorted reads to the output BAM file.
6.  **Execution Finished**:

    -   `[Mon Sep 23 05:25:56 UTC 2024] picard.sam.SortSam done. Elapsed time: 0.22 minutes.`
    -   This means the entire sorting process is complete, and it took around 0.22 minutes (about 13 seconds) to finish the task.
7.  **Memory Usage**:

    -   `Runtime.totalMemory()=122683392`
    -   This is the amount of memory (in bytes) allocated to the Java Virtual Machine (JVM) during the execution.

### Summary:

The log indicates that the `SortSam` command successfully sorted the input BAM file, despite encountering many non-increasing record positions (which is normal for unsorted BAM files). The new syntax notification is just a heads-up for future command usage. The operation was completed in around 13 seconds, and everything seems to have worked as expected.

In [ ]:
!java -jar picard.jar MarkDuplicates \
    INPUT=/content/sorted_reads.bam \
    OUTPUT=/content/dedup_reads.bam \
    METRICS_FILE=/content/duplication_metrics.txt

The log you've shared is the detailed output of running the `MarkDuplicates` command using Picard. Here's a step-by-step explanation of the information provided in the log:

Key Points in the Log:

1.  **Command Line Syntax Change Notification**:

    -   Similar to `SortSam`, Picard is informing you that the command line syntax is changing. The log provides the new format of the `MarkDuplicates` command:

        bash

        Copy code

        `MarkDuplicates -INPUT /content/sorted_reads.bam -OUTPUT /content/dedup_reads.bam -METRICS_FILE /content/duplication_metrics.txt`

2.  **Library Loading**:

    -   `INFO NativeLibraryLoader - Loading libgkl_compression.so...`
    -   Picard is loading native libraries for compression, which are required to handle BAM files efficiently.
3.  **Command Execution Details**:

    -   The command being executed is:

        javascript

        Copy code

        `MarkDuplicates INPUT=[/content/sorted_reads.bam] OUTPUT=/content/dedup_reads.bam METRICS_FILE=/content/duplication_metrics.txt ...`

    -   Key parameters used:
        -   `INPUT`: Specifies the input sorted BAM file.
        -   `OUTPUT`: Specifies the output BAM file with duplicates marked.
        -   `METRICS_FILE`: Specifies the file where duplication metrics (e.g., number of duplicates) are written.
        -   Various other parameters control aspects like memory usage, duplicate scoring, and how read names are parsed.
4.  **Start of Processing**:

    -   `INFO MarkDuplicates Start of doWork freeMemory: 24723776; totalMemory: 31457280; maxMemory: 3403677696`
    -   Shows initial memory allocation for the Java process before starting the duplicate marking operation.
5.  **Reading and Constructing Read End Information**:

    -   `INFO MarkDuplicates Reading input file and constructing read end information.`
    -   Picard is reading the input BAM file to gather information about the reads' ends (the start and end positions of each read) to identify duplicates.
6.  **Non-Integer Read Name Warning**:

    -   `WARNING AbstractOpticalDuplicateFinderCommandLineProgram ... Read name: SRR30683486.118524. Cause: String 'SRR30683486.118524' did not start with a parsable number.`
    -   This warning indicates that a part of a read name, which Picard expected to contain an integer, did not. This is usually not a critical issue but indicates that the read naming convention is not as expected by Picard.
7.  **Number of Records Read**:

    -   `INFO MarkDuplicates Read 568784 records. 0 pairs never matched.`
    -   Picard read 568,784 records from the BAM file and did not encounter any unmatched read pairs, which means all paired-end reads were properly aligned.
8.  **Memory Usage Information**:

    -   Several memory usage statistics are logged throughout the process, indicating how much memory Picard is using as it processes the file.
9.  **Duplicate Detection**:

    -   `INFO MarkDuplicates Traversing read pair information and detecting duplicates.`
    -   Picard is now checking each read pair for duplicates.
10. **Duplicates Marked**:

    -   `INFO MarkDuplicates Marking 102504 records as duplicates.`
    -   102,504 records were identified and marked as duplicates.
11. **Optical Duplicate Detection**:

    -   `INFO MarkDuplicates Found 0 optical duplicate clusters.`
    -   No optical duplicates (duplicates caused by the imaging system of the sequencer) were found. This is good, as it suggests no imaging artifacts.
12. **Writing Complete**:

    -   `INFO MarkDuplicates Writing complete. Closing input iterator.`
    -   Picard has finished writing the output BAM file with duplicates marked and is closing the input and output files.
13. **Duplicate Index Cleanup**:

    -   `INFO MarkDuplicates Duplicate Index cleanup.`
    -   Picard is cleaning up any temporary indices it created during the processing.
14. **Final Memory Statistics and Output Closure**:

    -   Shows the memory usage statistics before and after closing the output file.
15. **Execution Complete**:

    -   `[Mon Sep 23 05:49:11 UTC 2024] picard.sam.markduplicates.MarkDuplicates done. Elapsed time: 0.28 minutes.`
    -   The `MarkDuplicates` command has finished execution in approximately 0.28 minutes (about 17 seconds).
16. **Final Memory Usage**:

    -   `Runtime.totalMemory()=28311552`
    -   This shows the final memory allocated to the JVM after completing the command.

Summary:

The `MarkDuplicates` command successfully identified and marked duplicates in the BAM file. It read 568,784 records and marked 102,504 of them as duplicates. The process took around 17 seconds to complete, and no optical duplicates were found. The log also includes detailed information on memory usage and processing steps, indicating that everything completed successfully without critical errors.


In [ ]:
!cat  /content/duplication_metrics.txt


In [ ]:
!java -jar picard.jar SamFormatConverter \
    INPUT=/content/aligned_reads.bam \
    OUTPUT=/content/aligned_reads.sam

 Command Breakdown:

`!java -jar picard.jar AddOrReplaceReadGroups\
    INPUT=/content/aligned_reads.bam\
    OUTPUT=/content/rg_aligned_reads.bam\
    RGID=1\
    RGLB=lib1\
    RGPL=illumina\
    RGPU=unit1\
    RGSM=sample`

-   `!java -jar picard.jar AddOrReplaceReadGroups \`:

    -   This is the main command to run Picard's `AddOrReplaceReadGroups` tool using the Java command-line interface.
-   `INPUT=/content/aligned_reads.bam \`:

    -   Specifies the path to the input BAM file (`/content/aligned_reads.bam`) where the read group information will be added or replaced.
-   `OUTPUT=/content/rg_aligned_reads.bam \`:

    -   Specifies the path to the output BAM file (`/content/rg_aligned_reads.bam`) with the updated read group information.
-   `RGID=1 \`:

    -   `RGID` stands for "Read Group ID." It uniquely identifies a read group within the BAM file. Here, it is set to `1`.
-   `RGLB=lib1 \`:

    -   `RGLB` stands for "Read Group Library." It specifies the library preparation kit or process used. Here, it is set to `lib1`.
-   `RGPL=illumina \`:

    -   `RGPL` stands for "Read Group Platform." It specifies the sequencing platform used. In this case, `illumina` is specified, which is a commonly used sequencing technology.
-   `RGPU=unit1 \`:

    -   `RGPU` stands for "Read Group Platform Unit." It typically refers to the run or flow cell barcode used for sequencing. Here, it is set to `unit1`.
-   `RGSM=sample`:

    -   `RGSM` stands for "Read Group Sample." It specifies the name of the sample being sequenced. Here, it is set to `sample`.

 Why Is This Important?

1.  **Metadata Annotation**: Adding read group information helps in keeping track of different sequencing runs, libraries, and samples, which is crucial for organizing and analyzing complex datasets.

2.  **Downstream Analysis**: Many tools, like GATK (Genome Analysis Toolkit) for variant calling, require read group information to differentiate between data from different samples or sequencing runs within a single BAM file.

3.  **Quality Control**: By including details about the platform and library, researchers can perform quality checks and identify issues specific to a particular run or library preparation.

 Summary:

This command adds or updates metadata in your BAM file to include specific details about the sequencing process and sample. This information is essential for proper data management and accurate downstream analysis.

In [ ]:
!samtools view -h /content/dedup_reads.bam > /content/dedup_reads.sam

In [ ]:
!head /content/dedup_reads.sam

In [ ]:
!head -n 30 /content/dedup_reads.sam

# Base recalibration

Install GATK:

In [ ]:
!wget -c https://github.com/broadinstitute/gatk/releases/download/4.3.0.0/gatk-4.3.0.0.zip

In [ ]:
!unzip -o gatk-4.3.0.0.zip

Check GATK Installation:

In [ ]:
!gatk-4.3.0.0/gatk --version

If your reference genome is not indexed, you need to index it:

Generate Recalibration Table Using BaseRecalibrator. Use the following command to generate the recalibration table:

In [ ]:
# Replace 'M.tuberculosis_H37Rv' with 'AL123456' in the VCF file
!sed -i 's/M.tuberculosis_H37Rv/AL123456/g' "/content/drive/MyDrive/Academic/CMI DS 23-25/IP/Meeting 4: 23 09 2024/MTB_Base_Calibration_List.vcf"

Index the VCF File Using GATK:

Run the following command to create an index file for your VCF:

In [ ]:
!java -jar /content/gatk-4.3.0.0/gatk-package-4.3.0.0-local.jar IndexFeatureFile \
   -I "/content/drive/MyDrive/Academic/CMI DS 23-25/IP/Meeting 4: 23 09 2024/MTB_Base_Calibration_List.vcf"

Apply the Recalibration: Now you need to apply the recalibration to your BAM file to generate a new recalibrated BAM file. Use the ApplyBQSR tool as follows:

In [ ]:
!wget -c https://github.com/broadinstitute/picard/releases/download/2.27.4/picard.jar

!java -jar picard.jar AddOrReplaceReadGroups \
   I=/content/dedup_reads.bam \
   O=/content/rg_dedup_reads.bam \
   RGID=1 \
   RGLB=lib1 \
   RGPL=illumina \
   RGPU=unit1 \
   RGSM=sample

Index the rg_dedup_reads.bam bam file

In [ ]:
!samtools index /content/rg_dedup_reads.bam

This command is using the BaseRecalibrator tool from the Genome Analysis Toolkit (GATK) to perform base quality score recalibration (BQSR) on your BAM file. Here's a detailed breakdown of what each part of the command does:

In [ ]:
!java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /content/gatk-4.3.0.0/gatk-package-4.3.0.0-local.jar BaseRecalibrator \
   -I /content/rg_dedup_reads.bam \
   -R /content/mycobacterium_tuberculosis_h37rv_2_contigs.fasta \
   --known-sites "/content/drive/MyDrive/IP/Meeting 4: 23 09 2024/MTB_Base_Calibration_List.vcf" \
   -O /content/recal_data.table

Apply the Recalibration: Now you need to apply the recalibration to your BAM file to generate a new recalibrated BAM file. Use the ApplyBQSR tool as follows:

In [ ]:
! samtools faidx mycobacterium_tuberculosis_h37rv_2_contigs.fasta

In [ ]:
!java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /content/gatk-4.3.0.0/gatk-package-4.3.0.0-local.jar ApplyBQSR \
   -R /content/mycobacterium_tuberculosis_h37rv_2_contigs.fasta \
   -I /content/rg_dedup_reads.bam \
   --bqsr-recal-file /content/recal_data.table \
   -O /content/recalibrated_reads.bam

After applying the base recalibration using ApplyBQSR, your reads in the BAM file (recalibrated_reads.bam) are considered "analysis-ready." These reads have corrected base quality scores that better reflect the true accuracy of the sequencing data, making them suitable for downstream analysis such as variant calling.

In [ ]:
!samtools flagstat /content/recalibrated_reads.bam

Yes, this is good news! Let's break down the key information from the `samtools flagstat` output:

### Summary of the Output

1.  **Total Reads**:

    -   `574360 + 0 in total (QC-passed reads + QC-failed reads)`
    -   You have a total of 574,360 reads in your BAM file. All these reads passed the quality control (QC) filters.
2.  **Primary Alignments**:

    -   `573066 + 0 primary`
    -   There are 573,066 primary alignments. Primary alignments represent the main alignment of each read to the reference genome.
3.  **Supplementary Alignments**:

    -   `1294 + 0 supplementary`
    -   Supplementary alignments are typically used for reads that map to multiple locations or for chimeric alignments. You have 1,294 such alignments.
4.  **Duplicates**:

    -   `102504 + 0 duplicates`
    -   `102504 + 0 primary duplicates`
    -   You have 102,504 duplicate reads, which have been marked in your BAM file. These duplicates are expected as you performed duplicate marking earlier.
5.  **Mapped Reads**:

    -   `568784 + 0 mapped (99.03% : N/A)`
    -   `567490 + 0 primary mapped (99.03% : N/A)`
    -   99.03% of your reads are mapped to the reference genome, which is an excellent mapping rate. It indicates high-quality alignment.
6.  **Paired Reads**:

    -   `0 + 0 paired in sequencing`
    -   There are no paired-end reads, indicating your data is single-end.
7.  **Properly Paired Reads**:

    -   `0 + 0 properly paired (N/A : N/A)`
    -   This metric is not applicable for single-end reads.
8.  **Singletons and Mate Pairs**:

    -   `0 + 0 singletons (N/A : N/A)`
    -   `0 + 0 with mate mapped to a different chr`
    -   These values are also not applicable for single-end data.

### Interpretation

-   **High Mapping Rate (99.03%)**: This indicates that almost all your reads are successfully aligned to the reference genome, which is a strong indicator of data quality.
-   **Duplicates**: A significant number of duplicates (102,504 reads) is common in sequencing data and has been handled correctly by marking duplicates in your BAM file.
-   **No QC-Failed Reads**: All reads passed QC, which is good.

### Next Steps

1.  **Proceed with Variant Calling**: Since your reads are properly aligned and recalibrated, you can move on to variant calling or other downstream analyses.

2.  **Check for Specific Quality Issues**: If you want to ensure no specific issues are present, you can use additional tools like `samtools stats` for more in-depth statistics or visualize your reads in IGV.

Overall, the output looks good, and you are ready to proceed with further analysis. Let me know if you have any specific concerns or next steps in mind!